In [1]:
import pandas as pd
import re 
#making dictionary

In [2]:
#making dictionary

headings_data = pd.read_excel('GROUPED CLEANED HEADINGS.xlsx', sheet_name='Regex')
headings = headings_data.set_index('Format').T.to_dict('list')
headings

{'\\(D\\)-?': ['\\(D\\)-?',
  '\\(A\\)-? (?=Arrived to find|Patient found)?',
  '\\(C\\)-?',
  '\\((?:Hx|HPI|H)\\)-?',
  nan,
  nan,
  nan,
  '\\(A\\)-?',
  '\\((?:Rx|R)\\)-?',
  '\\(T\\)-?',
  '\\(E\\)-?',
  '\\(S\\)-?'],
 '(?<![A-Za-z0-9])d-': ['(?<![A-Za-z0-9])d-',
  '(?<![A-Za-z0-9])a- (?=arrived to find)?',
  '(?<![A-Za-z0-9])c-',
  '(?<![A-Za-z0-9])h-',
  nan,
  nan,
  nan,
  '(?<![A-Za-z0-9])a-',
  '(?<![A-Za-z0-9])r-',
  '(?<![A-Za-z0-9])t-',
  '(?<![A-Za-z0-9])e-',
  '(?<![A-Za-z0-9])s-'],
 'Dispatched Priority': ['Dispatched Priority',
  '\\[A\\]- ',
  '\\[C\\]-',
  '\\[H.P.I.\\]- ',
  nan,
  nan,
  nan,
  '\\[Assessment\\]',
  '\\[Rx\\]       ',
  '\\[Transport\\] - ',
  '\\[Extra, Exceptions\\]',
  'Signatures'],
 'SUBJECTIVE:': ['SUBJECTIVE:',
  'On arrival,',
  'Chief complaint of',
  nan,
  'Medical/Surgical History:',
  'Medication Allergies:',
  'Current Medications:',
  'OBJECTIVE:',
  'PLAN:',
  'The patient was transported to',
  nan,
  nan],
 '\\[Dispatch\\]': ['\\

In [3]:
col_labels = headings_data.drop(columns=['Format']).to_dict('list')

col_dict = {}

for key, value in col_labels.items():
    for each in value:
        col_dict[each] = key
col_labels.keys()

dict_keys(['Dispatch', 'Arrival', 'Chief Complaint', 'History of Present Illness', 'Medical/Surgical History', 'Medication Allergies', 'Current Medications', 'Assessment', 'Treatment', 'Transport', 'Exceptions/Extras', 'Signatures'])

In [4]:
#read in narrative data

data = pd.read_excel('COMBINED FINAL RAA Data, 2017-2020.xlsx')
data 

,ID,Priority,CallType,ChiefComplaint,Impression,Vitals,Procedures,Medications,Medic Notes,Narrative,Outcome
0,2020_2,1,Breathing Problem,"""""PAIN TO THE LEFT SIDE OF MY BODY""""""SOB""",{Pain - Chronic (G89.2)}{Hypertension (I10)}{R...,{2020-01-01T03:30:01: Pulse-96 Resp-16 BP-194/...,{2020-01-01T03:32:04: CV - ECG - 4 Lead Monito...,NaN,Arrived on scene to be welcomed inside the fr...,SUBJECTIVE: Called for a reported Breathing Pr...,"Patient Treated, Transported by this EMS Unit"
1,2020_3,1,Breathing Problem,"""Shortness of breath, nausea, vomiting, dizzin...",{Abuse of Alcohol - Intoxication (F10.92)}{Res...,{2020-01-01T02:28:17: Pulse-98 Resp-24 BP-109/...,{2020-01-01T02:44:10: CV - ECG - 4 Lead Monito...,NaN,Arrived to find a 29 y/o w/f sitting on her c...,SUBJECTIVE: Called for a reported Breathing Pr...,"Patient Treated, Transported by this EMS Unit"
2,2020_4,2,Falls,"""Mouth injury """,{Dental/Tooth Pain (K08.8)}{Injury - Face (S09...,{2020-01-01T02:05:41: Pulse-103 Resp-16 BP-123...,{2020-01-01T02:00:00: Behavior - Restraint App...,NaN,Arrived on scene to find pt being propped up ...,SUBJECTIVE: Called for a reported Falls at --A...,"Patient Treated, Transported by this EMS Unit"
3,2020_5,1,Breathing Problem,"""Abdominal pain""",{Pain - Abdominal Generalized (R10.84)}{Respir...,{2020-01-01T03:37:20: Pulse-102 Resp-18 BP-220...,{2020-01-01T03:38:23: CV - ECG - 4 Lead Monito...,{2020-01-01T03:38:00: Oxygen (7806)},Medic 91 dispatched Priority 1 Breathing Prob...,SUBJECTIVE: Called for a reported Breathing Pr...,"Patient Treated, Transported by this EMS Unit"
4,2020_6,1,Chest Pain (Non-Traumatic),"""Left chest pain""",{Hypertension (I10)}{Chest Pain - Cardiac (I20...,{2020-01-01T04:55:23: Pulse-115 Resp-16 BP-247...,{2020-01-01T04:52:43: CV - ECG - 4 Lead Monito...,{2020-01-01T05:00:20: Aspirin (ASA) (1191)},Medic 91 dispatched Priority 1 Chest Pain. RF...,SUBJECTIVE: Called for a reported Chest Pain (...,"Patient Treated, Transported by this EMS Unit"
...,...,...,...,...,...,...,...,...,...,...,...
35921,2017_7299,2,Unknown Problem,{Pain - Chest (Non-Cardiac)},{Pain - Chest (presumed non-cardiac) (R07.89)}...,{08:18:00: Pulse-98 Resp-12 BP-157/98 GCS-15 G...,{08:20:00: Cardiac Monitor (46825001)}{08:27:5...,{08:25:24: Aspirin (1191)},NaN,D: EMS 480 dispatched priority 2 for the Medic...,"Treated, Transported by EMS"
35922,2017_7300,2,Unknown Problem,{Pain - Hip},{Fever (R50.9)}{Pain - Chest (presumed non-car...,{16:07:14: Pulse-124 Resp-16 BP-143/99 GCS-15 ...,{16:09:00: Cardiac Monitor (46825001)}{16:09:4...,{16:07:00: Aspirin (1191)}{16:15:00: Ondansetr...,NaN,D - Medic 477 was dispatched for a medical ala...,"Treated, Transported by EMS"
35923,2017_7301,2,Unknown Problem,{Altered Consciousness - Disoriented - Diabeti...,{Endocrine - Hypoglycemia - Diabetic (E13.64)}...,{14:24:05: Pulse-91 Resp-20 BP-216/140 GCS-9 G...,{14:24:00: Cardiac Monitor (46825001)}{14:25:0...,{14:26:00: Dextrose 10% (237648)},NaN,Dispatched: Unknown Problem Units Responding...,"Treated, Transported by EMS"
35924,2017_7302,3,Unknown Problem,{Abdominal Pain - Generalized}{General Illness...,{GI/GU - Abdominal Generalized (R10.84)}{GI/GU...,{01:07:48: Pulse-80 Resp-16 BP-199/93 GCS-15 G...,{01:09:30: Cardiac Monitor (46825001)}{01:11:5...,{01:15:54: Ondansetron (Zofran) (26225)},NaN,D: Unit 587 dispatched priority 1 (downgraded ...,"Treated, Transported by EMS"


In [5]:
'''
how to separate narratives
    define new columns: Dispatch, Arrival, Chief Complaint, History of Present Illness, Assessment, Treatment, 
        Transport, Exceptions/Extras, Signatures, Medical/Surgical History, Medication Allergies, Current Medications

    for each key in the dictionary (each format of dispatch)
        see if the narrative contains that format
        if it does, split the narrative with the headings of that category
        
'''
import numpy as np

narratives = data['Narrative']


joined = data



sep = pd.DataFrame(np.empty((len(joined),len(col_labels.keys())), dtype= np.str), columns=col_labels.keys())

for dispatch_format in headings.keys():
    #print(dispatch_format)
    format_matching_narratives = narratives[narratives.str.contains(dispatch_format, regex=True, na=False)]
    remaining_narratives = format_matching_narratives
    for header in headings[dispatch_format]:
        #print(col_label)
        if not pd.isnull(header):
            heading_matching_narratives = remaining_narratives[remaining_narratives.str.contains(header, regex=True, na=False)] #see if header exists
            #print(heading_matching_narratives.head(2))
            print(header)
            split = heading_matching_narratives.str.split(pat=header, n=1, expand=True)
            #print(header, split.head(3))
            sep[header] = split[0]
            remaining_narratives = split[1]
    sep['end'] = remaining_narratives

joined = pd.concat([joined, sep], axis=1)


print(joined.columns)
joined


\(D\)-?
\(A\)-? (?=Arrived to find|Patient found)?
\(C\)-?
\((?:Hx|HPI|H)\)-?
\(A\)-?
\((?:Rx|R)\)-?
\(T\)-?
\(E\)-?
\(S\)-?
(?<![A-Za-z0-9])d-
(?<![A-Za-z0-9])a- (?=arrived to find)?
(?<![A-Za-z0-9])c-
(?<![A-Za-z0-9])h-
(?<![A-Za-z0-9])a-
(?<![A-Za-z0-9])r-
(?<![A-Za-z0-9])t-
(?<![A-Za-z0-9])e-
(?<![A-Za-z0-9])s-
Dispatched Priority
\[A\]- 
\[C\]-
\[H.P.I.\]- 
\[Assessment\]
\[Rx\]       
\[Transport\] - 
\[Extra, Exceptions\]
Signatures
SUBJECTIVE:
On arrival,
Chief complaint of
Medical/Surgical History:
Medication Allergies:
Current Medications:
OBJECTIVE:
PLAN:
The patient was transported to
\[Dispatch\]
\[Arrival\]
\[Chief Complaint\]
\[Hx of Present Illness\]
\[Initial Assessment\]
\[Treatment\]
\[Transport\]
\[Extra\]
\[Signatures\]
»DISPATCH-
»ARRIVAL-
»COMPLAINT-
»HISTORY-
»ASSESSMENT-
»Rx-
»TRANSPORT-
»EXCEPTIONS-
»SIGNATURES-
Dispatch:
Arrival:
CC:|Chief Complaint:|Chief complaint:
HPI:|History:
Assessment:|ASSESSMENT:
Rx:|Treatment(?: \(Rx\))?:
Transport:
Exceptions: 
Sign

,ID,Priority,CallType,ChiefComplaint,Impression,Vitals,Procedures,Medications,Medic Notes,Narrative,...,(?<![A-Za-z0-9])S(?::|\.|/|;|\)|-| -),A- Medic ?[0-9]{2},Arrived to find,C-,H-,A-,R-,T-,E-,S-
0,2020_2,1,Breathing Problem,"""""PAIN TO THE LEFT SIDE OF MY BODY""""""SOB""",{Pain - Chronic (G89.2)}{Hypertension (I10)}{R...,{2020-01-01T03:30:01: Pulse-96 Resp-16 BP-194/...,{2020-01-01T03:32:04: CV - ECG - 4 Lead Monito...,NaN,Arrived on scene to be welcomed inside the fr...,SUBJECTIVE: Called for a reported Breathing Pr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020_3,1,Breathing Problem,"""Shortness of breath, nausea, vomiting, dizzin...",{Abuse of Alcohol - Intoxication (F10.92)}{Res...,{2020-01-01T02:28:17: Pulse-98 Resp-24 BP-109/...,{2020-01-01T02:44:10: CV - ECG - 4 Lead Monito...,NaN,Arrived to find a 29 y/o w/f sitting on her c...,SUBJECTIVE: Called for a reported Breathing Pr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020_4,2,Falls,"""Mouth injury """,{Dental/Tooth Pain (K08.8)}{Injury - Face (S09...,{2020-01-01T02:05:41: Pulse-103 Resp-16 BP-123...,{2020-01-01T02:00:00: Behavior - Restraint App...,NaN,Arrived on scene to find pt being propped up ...,SUBJECTIVE: Called for a reported Falls at --A...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020_5,1,Breathing Problem,"""Abdominal pain""",{Pain - Abdominal Generalized (R10.84)}{Respir...,{2020-01-01T03:37:20: Pulse-102 Resp-18 BP-220...,{2020-01-01T03:38:23: CV - ECG - 4 Lead Monito...,{2020-01-01T03:38:00: Oxygen (7806)},Medic 91 dispatched Priority 1 Breathing Prob...,SUBJECTIVE: Called for a reported Breathing Pr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020_6,1,Chest Pain (Non-Traumatic),"""Left chest pain""",{Hypertension (I10)}{Chest Pain - Cardiac (I20...,{2020-01-01T04:55:23: Pulse-115 Resp-16 BP-247...,{2020-01-01T04:52:43: CV - ECG - 4 Lead Monito...,{2020-01-01T05:00:20: Aspirin (ASA) (1191)},Medic 91 dispatched Priority 1 Chest Pain. RF...,SUBJECTIVE: Called for a reported Chest Pain (...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35921,2017_7299,2,Unknown Problem,{Pain - Chest (Non-Cardiac)},{Pain - Chest (presumed non-cardiac) (R07.89)}...,{08:18:00: Pulse-98 Resp-12 BP-157/98 GCS-15 G...,{08:20:00: Cardiac Monitor (46825001)}{08:27:5...,{08:25:24: Aspirin (1191)},NaN,D: EMS 480 dispatched priority 2 for the Medic...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35922,2017_7300,2,Unknown Problem,{Pain - Hip},{Fever (R50.9)}{Pain - Chest (presumed non-car...,{16:07:14: Pulse-124 Resp-16 BP-143/99 GCS-15 ...,{16:09:00: Cardiac Monitor (46825001)}{16:09:4...,{16:07:00: Aspirin (1191)}{16:15:00: Ondansetr...,NaN,D - Medic 477 was dispatched for a medical ala...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35923,2017_7301,2,Unknown Problem,{Altered Consciousness - Disoriented - Diabeti...,{Endocrine - Hypoglycemia - Diabetic (E13.64)}...,{14:24:05: Pulse-91 Resp-20 BP-216/140 GCS-9 G...,{14:24:00: Cardiac Monitor (46825001)}{14:25:0...,{14:26:00: Dextrose 10% (237648)},NaN,Dispatched: Unknown Problem Units Responding...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35924,2017_7302,3,Unknown Problem,{Abdominal Pain - Generalized}{General Illness...,{GI/GU - Abdominal Generalized (R10.84)}{GI/GU...,{01:07:48: Pulse-80 Resp-16 BP-199/93 GCS-15 G...,{01:09:30: Cardiac Monitor (46825001)}{01:11:5...,{01:15:54: Ondansetron (Zofran) (26225)},NaN,D: Unit 587 dispatched priority 1 (downgraded ...,...,"Patient is NOT self pay, as noted in Billing ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#joined.to_excel('Unjoined narratives.xlsx')

In [7]:
#join the same headings together

import numpy as np

grouped = pd.DataFrame(np.empty((len(joined),len(col_labels.keys())), dtype= np.str), columns=col_labels.keys())



for key in col_labels.keys():
    #print('KEY', key)
    for f in col_labels[key]:
        if not pd.isnull(f):
            #print(f)
            to_merge = joined[f]
            if isinstance(to_merge, pd.Series): 
                to_merge = to_merge.reset_index(drop=True).to_frame()
            #print(to_merge)
            for (columnName, columnData) in to_merge.iteritems():
                #print(columnName)
                #print(grouped[key])
                #print(columnData)
                #print(grouped[key] + columnData)
                #grouped[key] = grouped[key] + columnData
                grouped[key] = grouped[key] + columnData.fillna('')
                #print(grouped[key])

grouped['end'] = joined['end']
grouped

,Dispatch,Arrival,Chief Complaint,History of Present Illness,Medical/Surgical History,Medication Allergies,Current Medications,Assessment,Treatment,Transport,Exceptions/Extras,Signatures,end
0,,Called for a reported Breathing Problem at --...,found a 51 year Female patient weighing 113 KG.,,"""PAIN TO THE LEFT SIDE OF MY BODY"". Secondary...","Blood - Hyperlipidema, CV - Primary Hypertens...","spironolactone (Aldactazide), hydrocodone (Lo...","lisinopril (Prinivil), Regular Insulin, Human...","At 03:22, the patient was found. Initial asse...",Treatments were administered as follows: 03:2...,,,NaN
1,,Called for a reported Breathing Problem at --...,found a 29 year Female.,,"Shortness of breath, nausea, vomiting, dizzin...",Behavior - Major Depression.,No Known Drug Allergy.,bupropion (Wellbutrin).,"At 02:25, the patient was found. Initial asse...",Treatments were administered as follows: 02:2...,,,NaN
2,,Called for a reported Falls at --Address Reda...,found a 22 year Female.,,Mouth injury . The patient's medical history...,CV - Primary Hypertension.,No Known Drug Allergy.,Ibuprofen [Motrin].,"At 01:50, the patient was found. Initial asse...",Treatments were administered as follows: 02:0...,,,NaN
3,,Called for a reported Breathing Problem at --...,found a 62 year Male.,,Abdominal pain. The patient's medical history...,Dependence on renal dialysis.,Unable to Complete.,Unable to Complete.,"At 03:27, the patient was found. Initial asse...",Treatments were administered as follows: 03:3...,,,NaN
4,,Called for a reported Chest Pain (Non-Traumat...,found a 53 year Male.,,Left chest pain. The patient's medical histor...,CV - Primary Hypertension.,Penicillin (PCN).,Hydrochlorothiazide HCTZ (Microzide).,"At 04:49, the patient was found. Initial asse...",Treatments were administered as follows: 04:5...,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35921,,EMS 480 dispatched priority 2 for the Medical...,,,,,,,,,,,NaN
35922,,Medic 477 was dispatched for a medical alarm....,"On arrival, pt was found sitting on the sofa ...",Chest pain.,,,,"Pt states she was lying down, and that she ex...","Pt has history of diabetes, hypertensions, an...",Pt was secured on stretcher in semi-fowler's ...,Pt was transported to Richmond Community Hosp...,,NaN
35923,,Unknown Problem Units Responding: 1 Suppo...,EMS witnessed FD on scene of manor residence....,"Altered sensorium, hypoglycemia",,,,,"Airway patent, equal chest rise and fall bila...",20g IV Catheter R A/C. Medications Adminis...,,,NaN
35924,,Unit 587 dispatched priority 1 (downgraded to...,Arrived on scene with RPD to find an adult ma...,"The patient complained of right flank pain, h...",,,,The patient reports that vomiting began appro...,General: Conscious. Alert and oriented to per...,The patient was placed on a continuous 4-lead...,All movements of the patient were made withou...,"Patient is NOT self pay, as noted in Billing ...",NaN


In [8]:
# extra = joined['end']
# to_add = pd.DataFrame(np.empty((len(joined),1), dtype= np.str), columns=['end'])
# for (columnName, columnData) in extra.iteritems():
#     to_add['end'] = to_add['end'].fillna('') + columnData.fillna('')
# grouped['end'] = to_add['end']

# grouped

In [9]:
#grouped.to_excel('Before renaming narratives.xlsx')

In [10]:
#rename columns
moved = grouped.copy()

#first, move all columns that have nan on left side.  
#search which rows are nan/blank onn HPI. replace with medical/surgical history
is_blank_HPI = moved['History of Present Illness'] ==''
moved['temp1']= moved['Medical/Surgical History'][is_blank_HPI]
#moved['History of Present Illness '] = moved['temp1']


#search which rows are blank in Medical/Surgical History. replace with Assessment
is_blank_med = moved['Medical/Surgical History'] ==''
moved['temp2']= moved['Assessment'][is_blank_med]
#moved['Medical/Surgical History'] = moved['temp2']

#combine temp arrays
moved['History of Present Illness'] = moved['History of Present Illness'].fillna('') + moved['temp1'].fillna('')
moved['Medical/Surgical History'] = moved['temp2'].fillna('')
moved.loc[moved['Current Medications'] == '', 'Assessment'] = ''
#drop temp arrays
moved = moved.drop(columns=['temp1', 'temp2'])

names = list(moved.columns)
print(names)


for i in range(len(names)-1):
    moved[names[i]] = moved[names[i+1]]

moved = moved.drop(columns=['end'])

moved

['Dispatch', 'Arrival', 'Chief Complaint', 'History of Present Illness', 'Medical/Surgical History', 'Medication Allergies', 'Current Medications', 'Assessment', 'Treatment', 'Transport', 'Exceptions/Extras', 'Signatures', 'end']


,Dispatch,Arrival,Chief Complaint,History of Present Illness,Medical/Surgical History,Medication Allergies,Current Medications,Assessment,Treatment,Transport,Exceptions/Extras,Signatures
0,Called for a reported Breathing Problem at --...,found a 51 year Female patient weighing 113 KG.,"""PAIN TO THE LEFT SIDE OF MY BODY"". Secondary...",,"Blood - Hyperlipidema, CV - Primary Hypertens...","spironolactone (Aldactazide), hydrocodone (Lo...","lisinopril (Prinivil), Regular Insulin, Human...","At 03:22, the patient was found. Initial asse...",Treatments were administered as follows: 03:2...,,,NaN
1,Called for a reported Breathing Problem at --...,found a 29 year Female.,"Shortness of breath, nausea, vomiting, dizzin...",,Behavior - Major Depression.,No Known Drug Allergy.,bupropion (Wellbutrin).,"At 02:25, the patient was found. Initial asse...",Treatments were administered as follows: 02:2...,,,NaN
2,Called for a reported Falls at --Address Reda...,found a 22 year Female.,Mouth injury . The patient's medical history...,,CV - Primary Hypertension.,No Known Drug Allergy.,Ibuprofen [Motrin].,"At 01:50, the patient was found. Initial asse...",Treatments were administered as follows: 02:0...,,,NaN
3,Called for a reported Breathing Problem at --...,found a 62 year Male.,Abdominal pain. The patient's medical history...,,Dependence on renal dialysis.,Unable to Complete.,Unable to Complete.,"At 03:27, the patient was found. Initial asse...",Treatments were administered as follows: 03:3...,,,NaN
4,Called for a reported Chest Pain (Non-Traumat...,found a 53 year Male.,Left chest pain. The patient's medical histor...,,CV - Primary Hypertension.,Penicillin (PCN).,Hydrochlorothiazide HCTZ (Microzide).,"At 04:49, the patient was found. Initial asse...",Treatments were administered as follows: 04:5...,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
35921,EMS 480 dispatched priority 2 for the Medical...,,,,,,,,,,,NaN
35922,Medic 477 was dispatched for a medical alarm....,"On arrival, pt was found sitting on the sofa ...",Chest pain.,"Pt states she was lying down, and that she ex...",,,,"Pt has history of diabetes, hypertensions, an...",Pt was secured on stretcher in semi-fowler's ...,Pt was transported to Richmond Community Hosp...,,NaN
35923,Unknown Problem Units Responding: 1 Suppo...,EMS witnessed FD on scene of manor residence....,"Altered sensorium, hypoglycemia",,,,,"Airway patent, equal chest rise and fall bila...",20g IV Catheter R A/C. Medications Adminis...,,,NaN
35924,Unit 587 dispatched priority 1 (downgraded to...,Arrived on scene with RPD to find an adult ma...,"The patient complained of right flank pain, h...",The patient reports that vomiting began appro...,,,,General: Conscious. Alert and oriented to per...,The patient was placed on a continuous 4-lead...,All movements of the patient were made withou...,"Patient is NOT self pay, as noted in Billing ...",NaN


In [11]:
moved = pd.concat([data, moved], axis=1)
moved

,ID,Priority,CallType,ChiefComplaint,Impression,Vitals,Procedures,Medications,Medic Notes,Narrative,...,Chief Complaint,History of Present Illness,Medical/Surgical History,Medication Allergies,Current Medications,Assessment,Treatment,Transport,Exceptions/Extras,Signatures
0,2020_2,1,Breathing Problem,"""""PAIN TO THE LEFT SIDE OF MY BODY""""""SOB""",{Pain - Chronic (G89.2)}{Hypertension (I10)}{R...,{2020-01-01T03:30:01: Pulse-96 Resp-16 BP-194/...,{2020-01-01T03:32:04: CV - ECG - 4 Lead Monito...,NaN,Arrived on scene to be welcomed inside the fr...,SUBJECTIVE: Called for a reported Breathing Pr...,...,"""PAIN TO THE LEFT SIDE OF MY BODY"". Secondary...",,"Blood - Hyperlipidema, CV - Primary Hypertens...","spironolactone (Aldactazide), hydrocodone (Lo...","lisinopril (Prinivil), Regular Insulin, Human...","At 03:22, the patient was found. Initial asse...",Treatments were administered as follows: 03:2...,,,NaN
1,2020_3,1,Breathing Problem,"""Shortness of breath, nausea, vomiting, dizzin...",{Abuse of Alcohol - Intoxication (F10.92)}{Res...,{2020-01-01T02:28:17: Pulse-98 Resp-24 BP-109/...,{2020-01-01T02:44:10: CV - ECG - 4 Lead Monito...,NaN,Arrived to find a 29 y/o w/f sitting on her c...,SUBJECTIVE: Called for a reported Breathing Pr...,...,"Shortness of breath, nausea, vomiting, dizzin...",,Behavior - Major Depression.,No Known Drug Allergy.,bupropion (Wellbutrin).,"At 02:25, the patient was found. Initial asse...",Treatments were administered as follows: 02:2...,,,NaN
2,2020_4,2,Falls,"""Mouth injury """,{Dental/Tooth Pain (K08.8)}{Injury - Face (S09...,{2020-01-01T02:05:41: Pulse-103 Resp-16 BP-123...,{2020-01-01T02:00:00: Behavior - Restraint App...,NaN,Arrived on scene to find pt being propped up ...,SUBJECTIVE: Called for a reported Falls at --A...,...,Mouth injury . The patient's medical history...,,CV - Primary Hypertension.,No Known Drug Allergy.,Ibuprofen [Motrin].,"At 01:50, the patient was found. Initial asse...",Treatments were administered as follows: 02:0...,,,NaN
3,2020_5,1,Breathing Problem,"""Abdominal pain""",{Pain - Abdominal Generalized (R10.84)}{Respir...,{2020-01-01T03:37:20: Pulse-102 Resp-18 BP-220...,{2020-01-01T03:38:23: CV - ECG - 4 Lead Monito...,{2020-01-01T03:38:00: Oxygen (7806)},Medic 91 dispatched Priority 1 Breathing Prob...,SUBJECTIVE: Called for a reported Breathing Pr...,...,Abdominal pain. The patient's medical history...,,Dependence on renal dialysis.,Unable to Complete.,Unable to Complete.,"At 03:27, the patient was found. Initial asse...",Treatments were administered as follows: 03:3...,,,NaN
4,2020_6,1,Chest Pain (Non-Traumatic),"""Left chest pain""",{Hypertension (I10)}{Chest Pain - Cardiac (I20...,{2020-01-01T04:55:23: Pulse-115 Resp-16 BP-247...,{2020-01-01T04:52:43: CV - ECG - 4 Lead Monito...,{2020-01-01T05:00:20: Aspirin (ASA) (1191)},Medic 91 dispatched Priority 1 Chest Pain. RF...,SUBJECTIVE: Called for a reported Chest Pain (...,...,Left chest pain. The patient's medical histor...,,CV - Primary Hypertension.,Penicillin (PCN).,Hydrochlorothiazide HCTZ (Microzide).,"At 04:49, the patient was found. Initial asse...",Treatments were administered as follows: 04:5...,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35921,2017_7299,2,Unknown Problem,{Pain - Chest (Non-Cardiac)},{Pain - Chest (presumed non-cardiac) (R07.89)}...,{08:18:00: Pulse-98 Resp-12 BP-157/98 GCS-15 G...,{08:20:00: Cardiac Monitor (46825001)}{08:27:5...,{08:25:24: Aspirin (1191)},NaN,D: EMS 480 dispatched priority 2 for the Medic...,...,,,,,,,,,,NaN
35922,2017_7300,2,Unknown Problem,{Pain - Hip},{Fever (R50.9)}{Pain - Chest (presumed non-car...,{16:07:14: Pulse-124 Resp-16 BP-143/99 GCS-15 ...,{16:09:00: Cardiac Monitor (46825001)}{16:09:4...,{16:07:00: Aspirin (1191)}{16:15:00: Ondansetr...,NaN,D - Medic 477 was dispatched for a medical ala...,...,Chest pain.,"Pt states she was lying down, and that she ex...",,,,"Pt has history of diabetes, hypertensions, an...",Pt 

In [12]:
moved.to_excel('FINAL Separated Narratives.xlsx')